In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
from scipy.optimize import fsolve
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import seaborn as sns
from statsmodels.api import *
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator,MultipleLocator
##################################################
def checker(date):
    if date.month in [1, 12]:
        return True
    return False

def get_year(date):
    return date.year

def to_float(x):
    x = float(x.replace('.','').replace(',', '.'))
    return x

def to_value(x):
    x = float(x[:-1].replace('.','').replace(',', '.'))*1000
    return x

def to_perc(x):
    return float(x[:-1].replace('.','').replace(',', '.'))/100

In [ ]:
databook = pd.read_excel('Ozon_Databook_Q3_24.xlsx', 'Public Databook', header = 4)
databook = databook.drop(list(range(0,4))+list(range(8,12))+list(range(15,18))+[21,28,29,30]+list(range(36,54)),axis = 0)
databook= databook.set_index('Unnamed: 0')

for i in range(4):
    databook.insert(i,f'Q{i+1}_18',databook['FY_2018']*(i+1)/10)

for i in range(4):
    databook.insert(i+4,f'Q{i+1}_19',databook['FY_2019']*(i+1)/10)

databook = databook.drop([i for i in databook.columns if 'F' in i]+['Unnamed: 23'],axis=1)

databook = databook.T
databook.columns.name = 'Quarter'
data_cols = databook.columns
databook

Quarter,GMV incl. services,"Share of Marketplace, as % of GMV incl. services","Number of orders, millions","Number of active buyers, millions",Total revenue,(Loss)/profit for the period,Adjusted EBITDA,Total non-current assets,Total current assets,Cash and cash equivalents,...,Total equity,Total non-current liabilities,Total current liabilities,Total liabilities,Total equity and liabilities,Movements in working capital1,Net cash (used in) / generated from operating activities1,Capital expenditures,Net cash (used in)/ generated from investing activities,Net cash(used in)/ generated from financing activities2
Q1_18,4188.818931,0.000871,1.528649,0.480000,3722.0,-566.1,-530.500000,646.8,1161.2,268.4,...,323.600,58.4,1426.0,1484.4,1808.000,151.7,-359.9,-256.5,-286.3,527.0
Q2_18,8377.637862,0.001742,3.057297,0.960000,7444.0,-1132.2,-1061.000000,1293.6,2322.4,536.8,...,647.200,116.8,2852.0,2968.8,3616.000,303.4,-719.8,-513.0,-572.6,1054.0
Q3_18,12566.456792,0.002614,4.585946,1.440000,11166.0,-1698.3,-1591.500000,1940.4,3483.6,805.2,...,970.800,175.2,4278.0,4453.2,5424.000,455.1,-1079.7,-769.5,-858.9,1581.0
Q4_18,16755.275723,0.003485,6.114594,1.920000,14888.0,-2264.4,-2122.000000,2587.2,4644.8,1073.6,...,1294.400,233.6,5704.0,5937.6,7232.000,606.8,-1439.6,-1026.0,-1145.2,2108.0
Q1_19,8081.486742,0.017378,3.181056,0.790000,6010.4,-1936.3,-1583.200000,1956.8,1886.7,300.3,...,81.700,811.2,2950.6,3761.8,3843.500,101.5,-1431.2,-476.8,-453.9,1933.5
Q2_19,16162.973484,0.034755,6.362112,1.580000,12020.8,-3872.6,-3166.400000,3913.6,3773.4,600.6,...,163.400,1622.4,5901.2,7523.6,7687.000,203.0,-2862.4,-953.6,-907.8,3867.0
Q3_19,24244.460225,0.052133,9.543168,2.370000,18031.2,-5808.9,-4749.600000,5870.4,5660.1,900.9,...,245.100,2433.6,8851.8,11285.4,11530.500,304.5,-4293.6,-1430.4,-1361.7,5800.5
Q4_19,32325.946967,0.069511,12.724224,3.160000,24041.6,-7745.2,-6332.800000,7827.2,7546.8,1201.2,...,326.800,3244.8,11802.4,15047.2,15374.000,406.0,-5724.8,-1907.2,-1815.6,7734.0
Q4_20,75847.502151,0.522714,29.610823,13.760362,37751.0,-9407.0,-3576.000000,29800.0,124808.0,103702.0,...,79257.000,15140.0,60211.0,75351.0,154608.000,15886.0,10644.0,-2059.0,-1966.0,91846.0
Q1_21,74208.000000,0.583574,34.100000,16.000000,33407.0,-6734.0,-4855.000000,33652.0,162747.0,140584.0,...,73273.000,71767.0,51359.0,123126.0,196399.000,-5890.0,-12118.0,-1955.0,-1597.0,47697.0


In [3]:
databook.columns = [i if i[len(i)-1]!=' ' else i[:-1] for i in databook.columns]
databook.columns = [i if i[len(i)-1]!=' ' else i[:-1] for i in databook.columns]
databook.columns

Index(['GMV incl. services',
       'Share of Marketplace, as % of GMV incl. services',
       'Number of orders, millions', 'Number of active buyers, millions',
       'Total revenue', '(Loss)/profit for the period', 'Adjusted EBITDA',
       'Total non-current assets', 'Total current assets',
       'Cash and cash equivalents', 'Total assets', 'Total equity',
       'Total non-current liabilities', 'Total current liabilities',
       'Total liabilities', 'Total equity and liabilities',
       'Movements in working capital1',
       'Net cash (used in) / generated from operating activities1',
       'Capital expenditures',
       'Net cash (used in)/ generated from investing activities',
       'Net cash(used in)/ generated from financing activities2'],
      dtype='object')

# Дополнения


In [4]:
databook['CCF'] = databook['Net cash (used in) / generated from operating activities1'] - databook['Capital expenditures']
databook['T'] = [0.2]*databook.shape[0]
databook['mu'] = databook['Adjusted EBITDA'] / databook['Total revenue']
databook['k_d'] = databook['Net cash(used in)/ generated from financing activities2']/databook['Total liabilities']
databook['D'] = databook['Total liabilities']
databook['S'] = databook['Total equity and liabilities'] - databook['D']
databook['L'] = databook['D']/databook['S']
databook['k_e'] = databook['mu'] 
databook['k_0'] = (databook['k_e'] + databook['L']*databook['k_d']*(1 - databook['T']))/(1+databook['L']*(1 - databook['T']))
databook['w_d'] = databook['D']/(databook['D'] + databook['S'])
databook

,GMV incl. services,"Share of Marketplace, as % of GMV incl. services","Number of orders, millions","Number of active buyers, millions",Total revenue,(Loss)/profit for the period,Adjusted EBITDA,Total non-current assets,Total current assets,Cash and cash equivalents,...,CCF,T,mu,k_d,D,S,L,k_e,k_0,w_d
Q1_18,4188.818931,0.000871,1.528649,0.480000,3722.0,-566.1,-530.500000,646.8,1161.2,268.4,...,-103.4,0.2,-0.142531,0.355026,1484.4,323.600,4.587145,-0.142531,0.248476,0.821018
Q2_18,8377.637862,0.001742,3.057297,0.960000,7444.0,-1132.2,-1061.000000,1293.6,2322.4,536.8,...,-206.8,0.2,-0.142531,0.355026,2968.8,647.200,4.587145,-0.142531,0.248476,0.821018
Q3_18,12566.456792,0.002614,4.585946,1.440000,11166.0,-1698.3,-1591.500000,1940.4,3483.6,805.2,...,-310.2,0.2,-0.142531,0.355026,4453.2,970.800,4.587145,-0.142531,0.248476,0.821018
Q4_18,16755.275723,0.003485,6.114594,1.920000,14888.0,-2264.4,-2122.000000,2587.2,4644.8,1073.6,...,-413.6,0.2,-0.142531,0.355026,5937.6,1294.400,4.587145,-0.142531,0.248476,0.821018
Q1_19,8081.486742,0.017378,3.181056,0.790000,6010.4,-1936.3,-1583.200000,1956.8,1886.7,300.3,...,-954.4,0.2,-0.263410,0.513983,3761.8,81.700,46.044064,-0.263410,0.493436,0.978743
Q2_19,16162.973484,0.034755,6.362112,1.580000,12020.8,-3872.6,-3166.400000,3913.6,3773.4,600.6,...,-1908.8,0.2,-0.263410,0.513983,7523.6,163.400,46.044064,-0.263410,0.493436,0.978743
Q3_19,24244.460225,0.052133,9.543168,2.370000,18031.2,-5808.9,-4749.600000,5870.4,5660.1,900.9,...,-2863.2,0.2,-0.263410,0.513983,11285.4,245.100,46.044064,-0.263410,0.493436,0.978743
Q4_19,32325.946967,0.069511,12.724224,3.160000,24041.6,-7745.2,-6332.800000,7827.2,7546.8,1201.2,...,-3817.6,0.2,-0.263410,0.513983,15047.2,326.800,46.044064,-0.263410,0.493436,0.978743
Q4_20,75847.502151,0.522714,29.610823,13.760362,37751.0,-9407.0,-3576.000000,29800.0,124808.0,103702.0,...,12703.0,0.2,-0.094726,1.218909,75351.0,79257.000,0.950717,-0.094726,0.472769,0.487368
Q1_21,74208.000000,0.583574,34.100000,16.000000,33407.0,-6734.0,-4855.000000,33652.0,162747.0,140584.0,...,-10163.0,0.2,-0.145329,0.387384,123126.0,73273.000,1.680373,-0.145329,0.160146,0.626918


In [ ]:
daya = [databook.loc['Q3_24','k_0'], databook.loc['Q3_24','k_d'], databook.loc['Q3_24','w_d'], databook.loc['Q3_24','T']]
def equation(wacc,n,f1):
    return (1-(1+wacc)**(-n))/wacc-f1


W, n = symbols('W, n')

f = (1-(1+daya[0])**(-n))/(daya[0]*(1-daya[2]*0.2 * (1-(1+daya[1])**(-n))))

N_ = []
waccs0 = []
for i in range(len(databook.index)):
    n_ = i + 1
    print(i,':',end='')
    sols = []
    if i:
        f1 = f.subs({n:n_})
        sols = [root.evalf() for root in solve((1-(1+W)**(-n_))/W-f1) if root.is_real and abs(root)<=1]
        waccs0.extend(sols)
   
    if sols==[]:
        waccs0.append(0)
        print(0)
    else:
        print(*sols)
    N_.append(n_)


fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MultipleLocator(1))
ax.plot(N_,waccs0)
ax.grid()
plt.title('WACC(n)')
plt.show()

0 :0
1 :0
2 :-0.0114557828387049
3 :-0.0118105058526054
4 :-0.0119838143173666
5 :-0.0120845971915875
6 :-0.0121491417792571
7 :-0.0121929974816194
8 :-0.0122239381740670
9 :-0.0122462822951223
10 :-0.0122626206921361
11 :-0.0122746019429316
12 :-0.0122833249926441
13 :-0.0122895505633434
14 :-0.0122938219547838
15 :-0.0122965375224938
16 :-0.0122979959753345
17 :-0.0122984256849868
18 :-0.0122980042252158
19 :-0.0122968717405938
20 :-0.0122951403044635
21 :-0.0122929006030959
22 :

In [ ]:
databook['WACC'] = waccs0


exog_cols  = databook.columns


# Определяем независимые переменные (X) и зависимую переменную (Y)
exog = databook[data_cols]

# Определяем независимые переменные (X) и зависимую переменную (Y)
'''scmn = MinMaxScaler()
scaler = StandardScaler()
exog = scaler.fit_transform(exog) '''


Y = databook['WACC']


# Строим модель с помощью метода наименьших квадратов (OLS)
model = OLS(Y,exog).fit()

# Выводим результаты модели
model.summary()

[[0.46 ]
 [0.462]
 [0.464]
 [0.466]
 [0.468]
 [0.47 ]
 [0.472]
 [0.474]
 [0.476]
 [0.478]
 [0.48 ]
 [0.482]
 [0.484]
 [0.486]
 [0.488]
 [0.49 ]
 [0.492]
 [0.494]
 [0.496]
 [0.498]
 [0.5  ]]


ValueError: endog and exog matrices are different sizes

In [ ]:
exog_cols  = databook.columns

# Определяем независимые переменные (X) и зависимую переменную (Y)
exog = databook[exog_cols]

# Определяем независимые переменные (X) и зависимую переменную (Y)
Y = databook['Number of orders, millions']


# Строим модель с помощью метода наименьших квадратов (OLS)
model = OLS(Y,exog).fit()

# Выводим результаты модели
model.summary()

In [ ]:
from scipy.stats import chi2
from itertools import combinations
from tqdm import tqdm

df = pd.DataFrame(exog)
df.columns = exog_cols

def get_outputs_icon(databook, stop_prints=True):
        
    # Шаг 1. Общий тест Хи-квадрат
    def chi_square_test(corr_matrix, n_obs):
        det = np.linalg.det(corr_matrix)
        if det <= 0:
            raise ValueError("Детерминант корреляционной матрицы отрицателен или равен нулю. Перепроверьте данные.")
        chi2_stat = -(n_obs - 1 - (2 / corr_matrix.shape[0])) * np.log(det)
        df = (corr_matrix.shape[0] * (corr_matrix.shape[0] - 1)) / 2
        p_value = chi2.sf(chi2_stat, df)
        return chi2_stat, p_value
    k=0
    try:
        corr_matrix = df.corr()
        n_obs = df.shape[0]
        chi2_stat, p_value = chi_square_test(corr_matrix, n_obs)
        if not stop_prints:
            print(f"Хи-квадрат статистика: {chi2_stat:.2f}, p-значение: {p_value:.4f}")

        # Интерпретация
        if p_value < 0.05:
            if not stop_prints:
                print("Обнаружена мультиколлинеарность (значимое отклонение от нуля).")
        else:
            if not stop_prints:
                print("Мультиколлинеарность не обнаружена.")

        # Шаг 2. Индивидуальный тест для каждой переменной
        from statsmodels.api import OLS, add_constant

        for col in df.columns:
            y = df[col]
            X = df.drop(columns=[col])
            X = add_constant(X)  # Добавляем константу для регрессии
            model = OLS(y, X).fit()
            f_stat = model.fvalue
            f_pval = model.f_pvalue
            if not stop_prints:
                print(f"\nТест для переменной {col}: F-статистика = {f_stat:.2f}, p-значение = {f_pval:.4f}")

            if f_pval < 0.05:
                if not stop_prints:
                    print(f"Переменная {col} находится в сильной корреляции с другими.")
            else:
                if not stop_prints:
                    print(f"Переменная {col} не вызывает значимой мультиколлинеарности.")

        # Шаг 3. Парные корреляции
        if not stop_prints:
            print("\nАнализ парной корреляции:")
        threshold = 0.8  # Порог для сильной корреляции
        for i in range(len(df.columns)):
            for j in range(i + 1, len(df.columns)):
                corr = corr_matrix.iloc[i, j]
                if not stop_prints:
                    print(f"Корреляция между {df.columns[i]} и {df.columns[j]}: {corr:.2f}")
                if abs(corr) > threshold:
                    if not stop_prints:
                        print(f"⚠️ Сильная корреляция между {df.columns[i]} и {df.columns[j]}!")
                    k+=1
        return k
    except:
        print('Возникла ошибка')
        return None

'''d = dict()
for i in tqdm(range(2,len(df.columns))):
    
    for j in tqdm(combinations(df.columns,i)):
        d[j] = 0
        print(get_outputs_icon(df[list(j)]))
        d[j] = get_outputs_icon(df[list(j)])'''

'd = dict()\nfor i in tqdm(range(2,len(df.columns))):\n    \n    for j in tqdm(combinations(df.columns,i)):\n        d[j] = 0\n        print(get_outputs_icon(df[list(j)]))\n        d[j] = get_outputs_icon(df[list(j)])'

Данные слишком мултиколлинеарны